<span style="font-size: 250%;color:white;background:green"> przykład ze internetu #3 BIS


https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0

<span style="font-size: 250%;color:green;background:#d9ead3">Zagadnienie transportowe

In [1]:
import pulp
from pulp import *
import pandas as pd
import numpy as np

### Definicja danych

Zdefiniujmy dane i przypiszmy je do zmiennych, które można następnie wykorzystać do wprowadzenia do modelu, funkcji celu i ograniczeń.

<span style="font-size: 150%;color:green;background:#d9ead3">Tworzę macierz

In [2]:
import pandas as pd

cc =["warehouse_1", "warehouse_2", "warehouse_3","warehouse_4", "warehouse_5", "warehouse_6","warehouse_7", "warehouse_8", "warehouse_9"]
aaa = [1, 3,1,1,3,4,3,1,2]
bbb = [2, 2,2,2,1,1,2,2,1]
ccc = [1, 1,1,3,4,4,3,1,4]
ddd = [1, 2,2,2,1,2,2,1,1]
eee = [3, 2,2,3,3,4,3,4,1]
fff = [1, 1,1,1,2,3,2,1,3]
ggg = [4, 2,1,2,1,3,1,2,3]
hhh = [1, 3,3,2,3,4,3,1,2]
kkk = [2, 2,3,2,1,4,1,1,3]
df = pd.DataFrame({'': cc, 'customer_1': aaa, 'customer_2': bbb, 'customer_3': ccc,'customer_4': ddd,'customer_5': eee, 'customer_6': fff,'customer_7': ggg, 'customer_8': hhh, 'customer_9': kkk})
df

,,customer_1,customer_2,customer_3,customer_4,customer_5,customer_6,customer_7,customer_8,customer_9
0,warehouse_1,1,2,1,1,3,1,4,1,2
1,warehouse_2,3,2,1,2,2,1,2,3,2
2,warehouse_3,1,2,1,2,2,1,1,3,3
3,warehouse_4,1,2,3,2,3,1,2,2,2
4,warehouse_5,3,1,4,1,3,2,1,3,1
5,warehouse_6,4,1,4,2,4,3,3,4,4
6,warehouse_7,3,2,3,2,3,2,1,3,1
7,warehouse_8,1,2,1,1,4,1,2,1,1
8,warehouse_9,2,1,4,1,1,3,3,2,3


In [3]:
n_warehouses, n_customers = df.shape

In [4]:
print('n_warehouses:',n_warehouses)
print('n_customers:',n_customers-1)   # odejmujemy kolumnę z  nazwami 'warehouse'

n_warehouses: 9
n_customers: 9


<span style="font-size: 150%;color:green;background:#d9ead3">Tworzę macierz Tworzę macierz kosztów transportu numpy

In [5]:
df.columns

Index(['', 'customer_1', 'customer_2', 'customer_3', 'customer_4',
       'customer_5', 'customer_6', 'customer_7', 'customer_8', 'customer_9'],
      dtype='object')

In [6]:
cost_matrix = df[['customer_1', 'customer_2', 'customer_3', 'customer_4','customer_5', 'customer_6', 'customer_7', 'customer_8', 'customer_9']]

cost_matrix = np.array(cost_matrix)  
cost_matrix

array([[1, 2, 1, 1, 3, 1, 4, 1, 2],
       [3, 2, 1, 2, 2, 1, 2, 3, 2],
       [1, 2, 1, 2, 2, 1, 1, 3, 3],
       [1, 2, 3, 2, 3, 1, 2, 2, 2],
       [3, 1, 4, 1, 3, 2, 1, 3, 1],
       [4, 1, 4, 2, 4, 3, 3, 4, 4],
       [3, 2, 3, 2, 3, 2, 1, 3, 1],
       [1, 2, 1, 1, 4, 1, 2, 1, 1],
       [2, 1, 4, 1, 1, 3, 3, 2, 3]])

In [7]:
cost_matrix.shape

(9, 9)

<span style="font-size: 150%;color:green;background:#d9ead3">Określam wektor zdolności dostarczania produktów przez hurtownie i zdolność sprzedaży przez sklepy

In [8]:
cust_demands = np.array([35000, 22000, 18000, 30000,32000, 21000, 11000, 31000,13000])

In [9]:
warehouse_supply = np.array([60000, 80000,40000, 30000, 62000, 17000,26000, 35000,22000])

<span style="font-size: 150%;color:black;background:#d9ead3">Inicjacja modelu
    
    Tworzę końcówki nazw zmiennych w macierzy zmiennych decyzyjnych

In [10]:
bis = LpProblem("Supply-Demand-Proble_bis", LpMinimize)

<span style="font-size: 150%;color:black;background:#d9ead3">Definiuje nazwy zmiennych (do macierzy)

In [11]:
variable_names = [str(i)+str(j) for j in range(1, n_customers) for i in range(1,n_warehouses+1)]
variable_names.sort()
#variable_names

<span style="font-size: 150%;color:black;background:#d9ead3">Zmienne decyzyjne
    
    Tworzę zmienne decyzyjne

In [12]:
DV_variables = LpVariable.matrix("X", variable_names , cat = "Integer" , lowBound= 0 )
#DV_variables

    Tworzę te zmienne decyzyjne w formacie macierzy

In [13]:
allocation = np.array(DV_variables).reshape(n_warehouses,n_customers-1)
allocation

array([[X_11, X_12, X_13, X_14, X_15, X_16, X_17, X_18, X_19],
       [X_21, X_22, X_23, X_24, X_25, X_26, X_27, X_28, X_29],
       [X_31, X_32, X_33, X_34, X_35, X_36, X_37, X_38, X_39],
       [X_41, X_42, X_43, X_44, X_45, X_46, X_47, X_48, X_49],
       [X_51, X_52, X_53, X_54, X_55, X_56, X_57, X_58, X_59],
       [X_61, X_62, X_63, X_64, X_65, X_66, X_67, X_68, X_69],
       [X_71, X_72, X_73, X_74, X_75, X_76, X_77, X_78, X_79],
       [X_81, X_82, X_83, X_84, X_85, X_86, X_87, X_88, X_89],
       [X_91, X_92, X_93, X_94, X_95, X_96, X_97, X_98, X_99]],
      dtype=object)

In [14]:
allocation.shape

(9, 9)

In [15]:
cost_matrix.shape

(9, 9)

<span style="font-size: 150%;color:black;background:#d9ead3">Definiuję funkcję celu

In [16]:
obj_func = lpSum(allocation*cost_matrix)
obj_func

1*X_11 + 2*X_12 + 1*X_13 + 1*X_14 + 3*X_15 + 1*X_16 + 4*X_17 + 1*X_18 + 2*X_19 + 3*X_21 + 2*X_22 + 1*X_23 + 2*X_24 + 2*X_25 + 1*X_26 + 2*X_27 + 3*X_28 + 2*X_29 + 1*X_31 + 2*X_32 + 1*X_33 + 2*X_34 + 2*X_35 + 1*X_36 + 1*X_37 + 3*X_38 + 3*X_39 + 1*X_41 + 2*X_42 + 3*X_43 + 2*X_44 + 3*X_45 + 1*X_46 + 2*X_47 + 2*X_48 + 2*X_49 + 3*X_51 + 1*X_52 + 4*X_53 + 1*X_54 + 3*X_55 + 2*X_56 + 1*X_57 + 3*X_58 + 1*X_59 + 4*X_61 + 1*X_62 + 4*X_63 + 2*X_64 + 4*X_65 + 3*X_66 + 3*X_67 + 4*X_68 + 4*X_69 + 3*X_71 + 2*X_72 + 3*X_73 + 2*X_74 + 3*X_75 + 2*X_76 + 1*X_77 + 3*X_78 + 1*X_79 + 1*X_81 + 2*X_82 + 1*X_83 + 1*X_84 + 4*X_85 + 1*X_86 + 2*X_87 + 1*X_88 + 1*X_89 + 2*X_91 + 1*X_92 + 4*X_93 + 1*X_94 + 1*X_95 + 3*X_96 + 3*X_97 + 2*X_98 + 3*X_99 + 0

Dodaję funkcję celu do zadania

In [17]:
bis +=  obj_func

In [18]:
bis

Supply-Demand-Proble_bis:
MINIMIZE
1*X_11 + 2*X_12 + 1*X_13 + 1*X_14 + 3*X_15 + 1*X_16 + 4*X_17 + 1*X_18 + 2*X_19 + 3*X_21 + 2*X_22 + 1*X_23 + 2*X_24 + 2*X_25 + 1*X_26 + 2*X_27 + 3*X_28 + 2*X_29 + 1*X_31 + 2*X_32 + 1*X_33 + 2*X_34 + 2*X_35 + 1*X_36 + 1*X_37 + 3*X_38 + 3*X_39 + 1*X_41 + 2*X_42 + 3*X_43 + 2*X_44 + 3*X_45 + 1*X_46 + 2*X_47 + 2*X_48 + 2*X_49 + 3*X_51 + 1*X_52 + 4*X_53 + 1*X_54 + 3*X_55 + 2*X_56 + 1*X_57 + 3*X_58 + 1*X_59 + 4*X_61 + 1*X_62 + 4*X_63 + 2*X_64 + 4*X_65 + 3*X_66 + 3*X_67 + 4*X_68 + 4*X_69 + 3*X_71 + 2*X_72 + 3*X_73 + 2*X_74 + 3*X_75 + 2*X_76 + 1*X_77 + 3*X_78 + 1*X_79 + 1*X_81 + 2*X_82 + 1*X_83 + 1*X_84 + 4*X_85 + 1*X_86 + 2*X_87 + 1*X_88 + 1*X_89 + 2*X_91 + 1*X_92 + 4*X_93 + 1*X_94 + 1*X_95 + 3*X_96 + 3*X_97 + 2*X_98 + 3*X_99 + 0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_15 Integer
0 <= X_16 Integer
0 <= X_17 Integer
0 <= X_18 Integer
0 <= X_19 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_2

<span style="font-size: 150%;color:black;background:#d9ead3">Warunki ograniczające
    
    Ograniczenia hurtowni

In [19]:
# Dodaję ograniczenia do zadania w pętli
# Warehouse Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers-1)) <= warehouse_supply[i])
    bis += lpSum(allocation[i][j] for j in range(n_customers-1)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

X_11 + X_12 + X_13 + X_14 + X_15 + X_16 + X_17 + X_18 + X_19 <= 60000
X_21 + X_22 + X_23 + X_24 + X_25 + X_26 + X_27 + X_28 + X_29 <= 80000
X_31 + X_32 + X_33 + X_34 + X_35 + X_36 + X_37 + X_38 + X_39 <= 40000
X_41 + X_42 + X_43 + X_44 + X_45 + X_46 + X_47 + X_48 + X_49 <= 30000
X_51 + X_52 + X_53 + X_54 + X_55 + X_56 + X_57 + X_58 + X_59 <= 62000
X_61 + X_62 + X_63 + X_64 + X_65 + X_66 + X_67 + X_68 + X_69 <= 17000
X_71 + X_72 + X_73 + X_74 + X_75 + X_76 + X_77 + X_78 + X_79 <= 26000
X_81 + X_82 + X_83 + X_84 + X_85 + X_86 + X_87 + X_88 + X_89 <= 35000
X_91 + X_92 + X_93 + X_94 + X_95 + X_96 + X_97 + X_98 + X_99 <= 22000


    Ograniczenia odbiorców

In [20]:
# Dodaję ograniczenia do zadania w pętli

for j in range(n_customers-1):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    bis += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

X_11 + X_21 + X_31 + X_41 + X_51 + X_61 + X_71 + X_81 + X_91 >= 35000
X_12 + X_22 + X_32 + X_42 + X_52 + X_62 + X_72 + X_82 + X_92 >= 22000
X_13 + X_23 + X_33 + X_43 + X_53 + X_63 + X_73 + X_83 + X_93 >= 18000
X_14 + X_24 + X_34 + X_44 + X_54 + X_64 + X_74 + X_84 + X_94 >= 30000
X_15 + X_25 + X_35 + X_45 + X_55 + X_65 + X_75 + X_85 + X_95 >= 32000
X_16 + X_26 + X_36 + X_46 + X_56 + X_66 + X_76 + X_86 + X_96 >= 21000
X_17 + X_27 + X_37 + X_47 + X_57 + X_67 + X_77 + X_87 + X_97 >= 11000
X_18 + X_28 + X_38 + X_48 + X_58 + X_68 + X_78 + X_88 + X_98 >= 31000
X_19 + X_29 + X_39 + X_49 + X_59 + X_69 + X_79 + X_89 + X_99 >= 13000


In [21]:
bis

Supply-Demand-Proble_bis:
MINIMIZE
1*X_11 + 2*X_12 + 1*X_13 + 1*X_14 + 3*X_15 + 1*X_16 + 4*X_17 + 1*X_18 + 2*X_19 + 3*X_21 + 2*X_22 + 1*X_23 + 2*X_24 + 2*X_25 + 1*X_26 + 2*X_27 + 3*X_28 + 2*X_29 + 1*X_31 + 2*X_32 + 1*X_33 + 2*X_34 + 2*X_35 + 1*X_36 + 1*X_37 + 3*X_38 + 3*X_39 + 1*X_41 + 2*X_42 + 3*X_43 + 2*X_44 + 3*X_45 + 1*X_46 + 2*X_47 + 2*X_48 + 2*X_49 + 3*X_51 + 1*X_52 + 4*X_53 + 1*X_54 + 3*X_55 + 2*X_56 + 1*X_57 + 3*X_58 + 1*X_59 + 4*X_61 + 1*X_62 + 4*X_63 + 2*X_64 + 4*X_65 + 3*X_66 + 3*X_67 + 4*X_68 + 4*X_69 + 3*X_71 + 2*X_72 + 3*X_73 + 2*X_74 + 3*X_75 + 2*X_76 + 1*X_77 + 3*X_78 + 1*X_79 + 1*X_81 + 2*X_82 + 1*X_83 + 1*X_84 + 4*X_85 + 1*X_86 + 2*X_87 + 1*X_88 + 1*X_89 + 2*X_91 + 1*X_92 + 4*X_93 + 1*X_94 + 1*X_95 + 3*X_96 + 3*X_97 + 2*X_98 + 3*X_99 + 0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 + X_15 + X_16 + X_17 + X_18
 + X_19 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 + X_25 + X_26 + X_27 + X_28
 + X_29 <= 80000

Supply_Constraints_2: X_31 + X_32 +

<span style="font-size: 150%;color:black;background:#d9ead3">Zapisuje problem

In [22]:
bis.writeLP("Supply_demand_prob.lp")

[X_11,
 X_12,
 X_13,
 X_14,
 X_15,
 X_16,
 X_17,
 X_18,
 X_19,
 X_21,
 X_22,
 X_23,
 X_24,
 X_25,
 X_26,
 X_27,
 X_28,
 X_29,
 X_31,
 X_32,
 X_33,
 X_34,
 X_35,
 X_36,
 X_37,
 X_38,
 X_39,
 X_41,
 X_42,
 X_43,
 X_44,
 X_45,
 X_46,
 X_47,
 X_48,
 X_49,
 X_51,
 X_52,
 X_53,
 X_54,
 X_55,
 X_56,
 X_57,
 X_58,
 X_59,
 X_61,
 X_62,
 X_63,
 X_64,
 X_65,
 X_66,
 X_67,
 X_68,
 X_69,
 X_71,
 X_72,
 X_73,
 X_74,
 X_75,
 X_76,
 X_77,
 X_78,
 X_79,
 X_81,
 X_82,
 X_83,
 X_84,
 X_85,
 X_86,
 X_87,
 X_88,
 X_89,
 X_91,
 X_92,
 X_93,
 X_94,
 X_95,
 X_96,
 X_97,
 X_98,
 X_99]

<span style="font-size: 150%;color:black;background:#d9ead3">Solve Problem and check Status

In [23]:
bis.solve()
status =  LpStatus[bis.status]
print(status)

Optimal


<span style="font-size: 150%;color:black;background:#d9ead3">Obliczenie wartości funkcji celu i zmiennych decyzyjnych

In [24]:
print("Total Cost:", bis.objective.value())
# Decision Variables

for v in bis.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 223000.0
X_11 = 0.0
X_12 = 0.0
X_13 = 0.0
X_14 = 0.0
X_15 = 0.0
X_16 = 0.0
X_17 = 0.0
X_18 = 0.0
X_19 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 0.0
X_24 = 0.0
X_25 = 10000.0
X_26 = 0.0
X_27 = 0.0
X_28 = 0.0
X_29 = 0.0
X_31 = 1000.0
X_32 = 0.0
X_33 = 18000.0
X_34 = 0.0
X_35 = 0.0
X_36 = 21000.0
X_37 = 0.0
X_38 = 0.0
X_39 = 0.0
X_41 = 30000.0
X_42 = 0.0
X_43 = 0.0
X_44 = 0.0
X_45 = 0.0
X_46 = 0.0
X_47 = 0.0
X_48 = 0.0
X_49 = 0.0
X_51 = 0.0
X_52 = 5000.0
X_53 = 0.0
X_54 = 30000.0
X_55 = 0.0
X_56 = 0.0
X_57 = 11000.0
X_58 = 0.0
X_59 = 13000.0
X_61 = 0.0
X_62 = 17000.0
X_63 = 0.0
X_64 = 0.0
X_65 = 0.0
X_66 = 0.0
X_67 = 0.0
X_68 = 0.0
X_69 = 0.0
X_71 = 0.0
X_72 = 0.0
X_73 = 0.0
X_74 = 0.0
X_75 = 0.0
X_76 = 0.0
X_77 = 0.0
X_78 = 0.0
X_79 = 0.0
X_81 = 4000.0
X_82 = 0.0
X_83 = 0.0
X_84 = 0.0
X_85 = 0.0
X_86 = 0.0
X_87 = 0.0
X_88 = 31000.0
X_89 = 0.0
X_91 = 0.0
X_92 = 0.0
X_93 = 0.0
X_94 = 0.0
X_95 = 22000.0
X_96 = 0.0
X_97 = 0.0
X_98 = 0.0
X_99 = 0.0


In [30]:
bis.variables

<bound method LpProblem.variables of Supply-Demand-Proble_bis:
MINIMIZE
1*X_11 + 2*X_12 + 1*X_13 + 1*X_14 + 3*X_15 + 1*X_16 + 4*X_17 + 1*X_18 + 2*X_19 + 3*X_21 + 2*X_22 + 1*X_23 + 2*X_24 + 2*X_25 + 1*X_26 + 2*X_27 + 3*X_28 + 2*X_29 + 1*X_31 + 2*X_32 + 1*X_33 + 2*X_34 + 2*X_35 + 1*X_36 + 1*X_37 + 3*X_38 + 3*X_39 + 1*X_41 + 2*X_42 + 3*X_43 + 2*X_44 + 3*X_45 + 1*X_46 + 2*X_47 + 2*X_48 + 2*X_49 + 3*X_51 + 1*X_52 + 4*X_53 + 1*X_54 + 3*X_55 + 2*X_56 + 1*X_57 + 3*X_58 + 1*X_59 + 4*X_61 + 1*X_62 + 4*X_63 + 2*X_64 + 4*X_65 + 3*X_66 + 3*X_67 + 4*X_68 + 4*X_69 + 3*X_71 + 2*X_72 + 3*X_73 + 2*X_74 + 3*X_75 + 2*X_76 + 1*X_77 + 3*X_78 + 1*X_79 + 1*X_81 + 2*X_82 + 1*X_83 + 1*X_84 + 4*X_85 + 1*X_86 + 2*X_87 + 1*X_88 + 1*X_89 + 2*X_91 + 1*X_92 + 4*X_93 + 1*X_94 + 1*X_95 + 3*X_96 + 3*X_97 + 2*X_98 + 3*X_99 + 0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 + X_15 + X_16 + X_17 + X_18
 + X_19 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 + X_25 + X_26 + X_27 + X_28
 + X_29 <= 80000

In [ ]:
cust_demands = np.array([35000, 22000, 18000, 30000,32000, 21000, 11000, 31000,13000])

<span style="font-size: 150%;color:black;background:#d9ead3"> Poziom wyjścia z hurtowni

In [25]:
for i in range(n_warehouses):
    print("Warehouse:", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers-1)))

Warehouse: 1
0.0
Warehouse: 2
10000.0
Warehouse: 3
40000.0
Warehouse: 4
30000.0
Warehouse: 5
59000.0
Warehouse: 6
17000.0
Warehouse: 7
0.0
Warehouse: 8
35000.0
Warehouse: 9
22000.0


In [26]:
warehouse_supply = np.array([60000, 80000,40000, 30000, 62000, 17000,26000, 35000,22000])

In [27]:
for i in range(n_customers-1):
    print("Customers:", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_warehouses)))

Customers: 1
0.0
Customers: 2
10000.0
Customers: 3
40000.0
Customers: 4
30000.0
Customers: 5
59000.0
Customers: 6
17000.0
Customers: 7
0.0
Customers: 8
35000.0
Customers: 9
22000.0


In [28]:
cust_demands = np.array([35000, 22000, 18000, 30000,32000, 21000, 11000, 31000,13000])